In [1]:
import sys
sys.path.append('../')
import daq
import file
import globalVAR as Gvar
import visa
import numpy as np
import time
from random import randint
import PyCapture2 as pc2

%load_ext autoreload
%autoreload 1

In [33]:
DAQ = daq.Daq(desc='Testing Daq handling of meta data.')
DAQ.connect_instr('KA3005P', 'ttyUSB0')
DAQ.connect_instr('TDS2024C', 1)

Power supply ID: KORADKA3005PV2.0
Device ttyUSB0 successfully connected and process started.
Oscilloscope ID: TEKTRONIX,TDS 2024C,C046401,CF:91.1CT FV:v24.26

Device C046401 successfully connected and process started.


/home/cu-pwfa/anaconda3/envs/CU-PWFA/lib/python3.6/site-packages/pyvisa/resources/messagebased.py:564: UserWarning: read string doesn't end with termination characters
  return self.read()


Scope Error: Empty preamble


Process Process-54:
Traceback (most recent call last):
  File "/home/cu-pwfa/anaconda3/envs/CU-PWFA/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/cu-pwfa/anaconda3/envs/CU-PWFA/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "../daq.py", line 221, in start_process
    proc_class(name, adr, c_queue, r_queue, o_queue)
  File "../processes/process.py", line 40, in __init__
    self.command_loop()
  File "../processes/process.py", line 68, in command_loop
    func(*com['args'])
  File "../processes/TDS2024C.py", line 23, in save_waveform
    t, y, pre = self.device.retrieve_current_waveform()
  File "../devices/TDS2024C.py", line 379, in retrieve_current_waveform
    if self.verify_format(pre) != True:
  File "../devices/TDS2024C.py", line 304, in verify_format
    elif pre["Byte number"] != 1:
KeyError: 'Byte number'
Process Process-51:
Traceback (most recent call last):
  Fil

In [34]:
DAQ.send_command(DAQ.command_queue['ttyUSB0'], 'turn_on')
DAQ.send_command(DAQ.command_queue['C046401'], 'acquire_off')
DAQ.send_command(DAQ.command_queue['C046401'], 'set_acquisition_stop', ("SEQuence",))
DAQ.send_command(DAQ.command_queue['C046401'], 'set_acquisition_mode', ("AVErage",))
DAQ.send_command(DAQ.command_queue['C046401'], 'set_average_num', (4,))

In [35]:
v = np.linspace(2.0, 30, 57)
chan = {'CH1' : [False, 'Floating Potential in Lab'],
        'CH2' : [False, 'Power Supply Setting']}

#for i in range(len(v)):
for j in [1,2]:
    ch = 'CH{}'.format(j)
    chan[ch][0] = True

    DAQ.send_command(DAQ.command_queue['C046401'], 'set_data_source', (ch,))
    time.sleep(3)
    
    DAQ.send_command(DAQ.command_queue['ttyUSB0'], 'set_record_voltage', (30,))
    time.sleep(0.1)
    
    DAQ.send_command(DAQ.command_queue['C046401'], 'acquire_on')
    time.sleep(1)
    
    DAQ.send_command(DAQ.command_queue['C046401'], 'save_waveform', (chan,))
    time.sleep(3)
    
    chan[ch][0] = False
        

KeyboardInterrupt: 

In [5]:
DAQ.end_dataset()

In [36]:
for key in DAQ.instr:
    for elem in DAQ.instr[key]:
        DAQ.disconnect_instr(key, elem)